Import Library

In [ ]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Gradcam support tools
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [ ]:
import random

SEED = 123

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

session_conf = tf.compat.v1.ConfigProto( intra_op_parallelism_threads=1, inter_op_parallelism_threads=1 )
sess = tf.compat.v1.Session( graph=tf.compat.v1.get_default_graph(), config=session_conf )
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Baca API Kaggle & dataset dari kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json

#liat kaggle.json nya dari sini fi : https://www.kaggle.com/settings/account terus pilih API-create new token

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!pip install kaggle

^C


In [ ]:
import kaggle

# Replace 'username/dataset' with the actual Kaggle dataset you want to download
dataset_name = 'rizqiakbarr13/larva-bit-to-high'

# Authenticate with Kaggle API
kaggle.api.authenticate()

# Download the dataset
kaggle.api.dataset_download_files(dataset_name, unzip=True)

##Membaca directory dari dataset nya

In [ ]:
data_dir = r'/content/larva'

## Membaca path data

In [ ]:
# Generate data paths with labels
def define_paths(data_dir):
    filepaths = []
    labels = []

    folds = os.listdir(data_dir)
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)

        # check the folders from main directory. If there are another files, ignore them
        if pathlib.Path(foldpath).suffix != '':
            continue

        filelist = os.listdir(foldpath)

        for file in filelist:
            fpath = os.path.join(foldpath, file)

            filepaths.append(fpath)
            labels.append(fold)

    return filepaths, labels

### Menyimpan path dalam dataframe

In [ ]:
# Concatenate data paths with labels into one dataframe ( to later be fitted into the model )
def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
files, classes = define_paths(data_dir)
df = define_df(files, classes)
df

,filepaths,labels
0,/content/larva/INSTAR4/images,INSTAR4
1,/content/larva/INSTAR3/images,INSTAR3
2,/content/larva/INSTAR1/images,INSTAR1
3,/content/larva/INSTAR2/images,INSTAR2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filepaths  4 non-null      object
 1   labels     4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [ ]:
df.labels.value_counts()

INSTAR4    1
INSTAR3    1
INSTAR1    1
INSTAR2    1
Name: labels, dtype: int64

## Membagi data (train, validation, test)

In [ ]:
import os

directory_path = '/content/train_data'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
else:
    print(f"The directory '{directory_path}' already exists.")


The directory '/content/train_data' already exists.


In [ ]:
import os
import shutil
import random

# Tentukan path dataset,lu buat pathh di rectory nya di folder content
data_dir = '/content/larva/INSTAR3/images'

# Daftar nama kelas (misalnya: 'kelas_1', 'kelas_2', dst.)
class_names = os.listdir(data_dir)

# Persentase pembagian data (misal: 70% train, 15% validation, 15% test)
train_split = 0.7
val_split = 0.15
test_split = 0.15

# List semua file dalam direktori data
file_list = os.listdir(data_dir)

# Mengacak urutan file
random.shuffle(file_list)

# Hitung jumlah file untuk setiap bagian
total_files = len(file_list)
train_count = int(total_files * train_split)
val_count = int(total_files * val_split)

# Memisahkan file menjadi train, validation, dan test
train_files = file_list[:train_count]
val_files = file_list[train_count:train_count + val_count]
test_files = file_list[train_count + val_count:]

# Membuat direktori baru jika belum ada
#bikin file train_dir, val_dir dan test_dir di folder content fi
train_dir = '/content/train_dir'
val_dir = '/content/val_dir'
test_dir = '/content/test_dir'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Memindahkan file ke direktori masing-masing
def move_files(files, source_dir, dest_dir):
    for file in files:
        src_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, file)
        shutil.move(src_path, dest_path)

move_files(train_files, data_dir, train_dir)
move_files(val_files, data_dir, val_dir)
move_files(test_files, data_dir, test_dir)

print("Data telah berhasil dibagi menjadi train, validation, dan test.")

Data telah berhasil dibagi menjadi train, validation, dan test.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Diasumsikan Anda sudah mendefinisikan train_dir, valid_dir, dan test_dir

# Membuat instance ImageDataGenerator untuk data train, validasi, dan test
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Menyiapkan generator menggunakan flow_from_directory
batch_size = 32
image_height = 150
image_width = 150

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Periksa panjang generator
print("Train Generator Length:", len(train_generator))
print("Validation Generator Length:", len(valid_generator))
print("Test Generator Length:", len(test_generator))


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Train Generator Length: 0
Validation Generator Length: 0
Test Generator Length: 0


In [ ]:
!pip install Keras-Preprocessing

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical

# Tentukan direktori tempat gambar-gambar Anda disimpan
data_directory = '/content/larva/INSTAR1/images/'

# Inisialisasi list untuk menyimpan data gambar dan label
X = []
y = []

# Loop melalui setiap gambar dalam direktori data
for image_file in os.listdir(data_directory):
    # Baca gambar dari direktori
    image_path = os.path.join(data_directory, image_file)
    image = load_img(image_path, target_size=(224, 224))  # Ubah ukuran gambar jika diperlukan
    image_array = img_to_array(image)
    X.append(image_array)

    # Encode label kelas
    # Jika nama gambar mengandung label, Anda bisa ekstrak label dari nama file.
    # Misalnya, jika nama file adalah 'cat001.jpg', labelnya adalah 'cat'.
    label = image_file.split('.')[0]  # Mengambil bagian sebelum ekstensi file (labelnya)
    y.append(label)

# Ubah list ke dalam array NumPy
X = np.array(X)
y = np.array(y)

# Normalisasi gambar ke dalam rentang 0-1
X = X / 255.0

# Encode label ke dalam bentuk angka menggunakan one-hot encoding
# Jika Anda sudah memiliki label dalam bentuk angka, Anda bisa melewati langkah ini
unique_labels = np.unique(y)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
y_encoded = np.array([label_to_index[label] for label in y])
y_encoded = to_categorical(y_encoded)

# Sekarang, Anda memiliki data `X` (gambar) dan `y_encoded` (label) siap untuk digunakan dalam model computer vision.



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np

# Misalkan X adalah data fitur dan y adalah label/target

# Bagi data menjadi data latih (train) dan data uji (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat instance StratifiedKFold dengan n_splits yang lebih rendah (misalnya, 3 lipatan)
n_splits = 2
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Inisialisasi list untuk menyimpan hasil akurasi dari setiap lipatan
fold_accuracies = []

# Lakukan validasi silang
for train_index, val_index in stratkf.split(X_train, y_train):
    # Bagi data latih dan validasi berdasarkan indeks yang dihasilkan oleh StratifiedKFold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Lakukan pelatihan model pada data latih (X_train_fold, y_train_fold)
    # Evaluasi model pada data validasi (X_val_fold, y_val_fold)
    # Simpan akurasi dari setiap lipatan ke dalam list fold_accuracies

# Setelah selesai validasi silang, hitung rata-rata akurasi dari semua lipatan
mean_accuracy = np.mean(fold_accuracies)

# Latih model final dengan data latih dan uji
# Evaluasi model pada data uji (X_test, y_test)


ValueError: ignored

In [ ]:
strat = df['labels']
train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 123, stratify= strat)

# valid and test dataframe
strat = dummy_df['labels']
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= strat)

In [ ]:
print(train_dir.labels.value_counts())
print(valid_dir.labels.value_counts())
print(test_dir.labels.value_counts())

## Pre-proses load data
### penentuan parameter data generator

In [ ]:
# define model parameters
img_size = (256, 256)
channels = 3 # either BGR or Grayscale
color = 'rgb'
img_shape = (img_size[0], img_size[1], channels)

# Recommended : use custom function for test data batch size, else we can use normal batch size.
# mencari faktor dari jumlah test set yang kurang dari sama dengan 80
ts_length = len(test_dir)
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

In [ ]:
test_batch_size

In [ ]:
sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80])

In [ ]:
test_steps

### Data Generator

In [ ]:
# train and validation batch size
batch_size = 64

# This function which will be used in image data generator for data augmentation, it just take the image and return it again.
def scalar(img):
    return img

tr_gen = ImageDataGenerator(preprocessing_function= scalar,
                            rescale=1./255,
                            zoom_range=0.5,
                            rotation_range=10,
                            horizontal_flip=True,
                            shear_range=0.2,
                            fill_mode='nearest')
ts_gen = ImageDataGenerator(preprocessing_function= scalar, rescale=1./255)

train_gen = tr_gen.flow_from_dataframe( train_dir, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= color, shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_dir, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= color, shuffle= True, batch_size= batch_size)

# Note: we will use custom test_batch_size, and make shuffle= false
test_gen = ts_gen.flow_from_dataframe( test_dir, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= color, shuffle= False, batch_size= test_batch_size)

In [ ]:
def show_images(gen):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = gen.class_indices        # defines dictionary {'class': index}
    classes = list(g_dict.keys())     # defines list of dictionary's kays (classes), classes names : string
    images, labels = next(gen)        # get a batch size samples from the generator

    # calculate number of displayed samples
    length = len(labels)        # length of batch size
    sample = min(length, 25)    # check if sample less than 25 images

    plt.figure(figsize= (20, 20))

    for i in range(sample):
        plt.subplot(5, 5, i + 1)
        image = images[i] #/ 255       # scales data to range (0 - 255)
        plt.imshow(image)
        index = np.argmax(labels[i])  # get image index
        class_name = classes[index]   # get class of image
        plt.title(class_name, color= 'blue', fontsize= 12)
        plt.axis('off')
    plt.show()

In [ ]:
show_images(train_gen)

In [ ]:
show_images(valid_gen)

In [ ]:
show_images(test_gen)

## Pengaturan Training Model
### Callback

In [ ]:
class MyCallback(keras.callbacks.Callback):
    def __init__(self, model, patience, stop_patience, threshold, factor, batches, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        self.model = model
        self.patience = patience # specifies how many epochs without improvement before learning rate is adjusted
        self.stop_patience = stop_patience # specifies how many times to adjust lr without improvement to stop training
        self.threshold = threshold # specifies training accuracy threshold when lr will be adjusted based on validation loss
        self.factor = factor # factor by which to reduce the learning rate
        self.batches = batches # number of training batch to run per epoch
        self.epochs = epochs
        self.ask_epoch = ask_epoch
        self.ask_epoch_initial = ask_epoch # save this value to restore if restarting training

        # callback variables
        self.count = 0 # how many times lr has been reduced without improvement
        self.stop_count = 0
        self.best_epoch = 1   # epoch with the lowest loss
        self.initial_lr = float(tf.keras.backend.get_value(model.optimizer.lr)) # get the initial learning rate and save it
        self.highest_tracc = 0.0 # set highest training accuracy to 0 initially
        self.lowest_vloss = np.inf # set lowest validation loss to infinity initially
        self.best_weights = self.model.get_weights() # set best weights to model's initial weights
        self.initial_weights = self.model.get_weights()   # save initial weights if they have to get restored

    # Define a function that will run when train begins
    def on_train_begin(self, logs= None):
        msg = 'Do you want model asks you to halt the training [y/n] ?'
        print(msg)
        ans = input('')
        if ans in ['Y', 'y']:
            self.ask_permission = 1
        elif ans in ['N', 'n']:
            self.ask_permission = 0

        msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format('Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor','% Improv', 'Duration')
        print(msg)
        self.start_time = time.time()


    def on_train_end(self, logs= None):
        stop_time = time.time()
        tr_duration = stop_time - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))

        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print(msg)

        # set the weights of the model to the best weights
        self.model.set_weights(self.best_weights)


    def on_train_batch_end(self, batch, logs= None):
        # get batch accuracy and loss
        acc = logs.get('accuracy') * 100
        loss = logs.get('loss')

        # prints over on the same line to show running batch count
        msg = '{0:20s}processing batch {1:} of {2:5s}-   accuracy=  {3:5.3f}   -   loss: {4:8.5f}'.format(' ', str(batch), str(self.batches), acc, loss)
        print(msg, '\r', end= '')


    def on_epoch_begin(self, epoch, logs= None):
        self.ep_start = time.time()


    # Define method runs on the end of each epoch
    def on_epoch_end(self, epoch, logs= None):
        ep_end = time.time()
        duration = ep_end - self.ep_start

        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
        current_lr = lr
        acc = logs.get('accuracy')  # get training accuracy
        v_acc = logs.get('val_accuracy')  # get validation accuracy
        loss = logs.get('loss')  # get training loss for this epoch
        v_loss = logs.get('val_loss')  # get the validation loss for this epoch

        if acc < self.threshold: # if training accuracy is below threshold adjust lr based on training accuracy
            monitor = 'accuracy'
            if epoch == 0:
                pimprov = 0.0
            else:
                pimprov = (acc - self.highest_tracc ) * 100 / self.highest_tracc # define improvement of model progres

            if acc > self.highest_tracc: # training accuracy improved in the epoch
                self.highest_tracc = acc # set new highest training accuracy
                self.best_weights = self.model.get_weights() # training accuracy improved so save the weights
                self.count = 0 # set count to 0 since training accuracy improved
                self.stop_count = 0 # set stop counter to 0
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1  # set the value of best epoch for this epoch

            else:
                # training accuracy did not improve check if this has happened for patience number of epochs
                # if so adjust learning rate
                if self.count >= self.patience - 1: # lr should be adjusted
                    lr = lr * self.factor # adjust the learning by factor
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) # set the learning rate in the optimizer
                    self.count = 0 # reset the count to 0
                    self.stop_count = self.stop_count + 1 # count the number of consecutive lr adjustments
                    self.count = 0 # reset counter
                    if v_loss < self.lowest_vloss:
                        self.lowest_vloss = v_loss
                else:
                    self.count = self.count + 1 # increment patience counter

        else: # training accuracy is above threshold so adjust learning rate based on validation loss
            monitor = 'val_loss'
            if epoch == 0:
                pimprov = 0.0

            else:
                pimprov = (self.lowest_vloss - v_loss ) * 100 / self.lowest_vloss

            if v_loss < self.lowest_vloss: # check if the validation loss improved
                self.lowest_vloss = v_loss # replace lowest validation loss with new validation loss
                self.best_weights = self.model.get_weights() # validation loss improved so save the weights
                self.count = 0 # reset count since validation loss improved
                self.stop_count = 0
                self.best_epoch = epoch + 1 # set the value of the best epoch to this epoch

            else: # validation loss did not improve
                if self.count >= self.patience - 1: # need to adjust lr
                    lr = lr * self.factor # adjust the learning rate
                    self.stop_count = self.stop_count + 1 # increment stop counter because lr was adjusted
                    self.count = 0 # reset counter
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) # set the learning rate in the optimizer

                else:
                    self.count = self.count + 1 # increment the patience counter

                if acc > self.highest_tracc:
                    self.highest_tracc = acc

        msg = f'{str(epoch + 1):^3s}/{str(self.epochs):4s} {loss:^9.3f}{acc * 100:^9.3f}{v_loss:^9.5f}{v_acc * 100:^9.3f}{current_lr:^9.5f}{lr:^9.5f}{monitor:^11s}{pimprov:^10.2f}{duration:^8.2f}'
        print(msg)

        if self.stop_count > self.stop_patience - 1: # check if learning rate has been adjusted stop_count times with no improvement
            msg = f' training has been halted at epoch {epoch + 1} after {self.stop_patience} adjustments of learning rate with no improvement'
            print(msg)
            self.model.stop_training = True # stop training

        else:
            if self.ask_epoch != None and self.ask_permission != 0:
                if epoch + 1 >= self.ask_epoch:
                    msg = 'enter H to halt training or an integer for number of epochs to run then ask again'
                    print(msg)

                    ans = input('')
                    if ans == 'H' or ans == 'h':
                        msg = f'training has been halted at epoch {epoch + 1} due to user input'
                        print(msg)
                        self.model.stop_training = True # stop training

                    else:
                        try:
                            ans = int(ans)
                            self.ask_epoch += ans
                            msg = f' training will continue until epoch {str(self.ask_epoch)}'
                            print(msg)
                            msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format('Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor', '% Improv', 'Duration')
                            print(msg)

                        except Exception:
                            print('Invalid')

In [ ]:
def plot_training(hist):
    '''
    This function take training model and plot history of accuracy and losses with the best epoch in both of them.
    '''

    # Define needed variables
    tr_acc = hist.history['accuracy']
    tr_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'

    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes, normalize= False, title= 'Confusion Matrix', cmap= plt.cm.Blues):
	'''
	This function plot confusion matrix method from sklearn package.
	'''

	plt.figure(figsize= (10, 10))
	plt.imshow(cm, interpolation= 'nearest', cmap= cmap)
	plt.title(title)
	plt.colorbar()

	tick_marks = np.arange(len(classes))
	plt.xticks(tick_marks, classes, rotation= 45)
	plt.yticks(tick_marks, classes)

	if normalize:
		cm = cm.astype('float') / cm.sum(axis= 1)[:, np.newaxis]
		print('Normalized Confusion Matrix')

	else:
		print('Confusion Matrix, Without Normalization')

	print(cm)

	thresh = cm.max() / 2.
	for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
		plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

	plt.tight_layout()
	plt.ylabel('True Label')
	plt.xlabel('Predicted Label')

## Train a model using Functional API
### Model VGG19 (untuk fine tuning)

In [ ]:
# Create Model Structure
img_size = (256, 256)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

# create pre-trained model (you can built on pretrained model such as :  efficientnet, VGG , Resnet )
# we will use efficientnetb3 from EfficientNet family.
base_model = tf.keras.applications.vgg19.VGG19(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')

# additional layer
x = Flatten()(base_model.output)
x = Dropout(rate=0.2)(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(class_count, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, outputs)

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])


In [ ]:
model.summary()

## Train the Functional API Model

### HyperParameter untuk training model

In [ ]:
batch_size = 64   # set batch size for training
epochs = 100   # number of all epochs in training
patience = 3   #number of epochs to wait to adjust lr if monitored value does not improve
stop_patience = 10   # number of epochs to wait before stopping training if monitored value does not improve
threshold = 0.9   # if train accuracy is < threshold adjust monitor accuracy, else monitor validation loss
factor = 0.5   # factor to reduce lr by
ask_epoch = 5   # number of epochs to run before asking if you want to halt training
batches = int(np.ceil(len(train_gen.labels) / batch_size))    # number of training batch to run per epoch

callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

In [ ]:
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

## graphic plotting

In [ ]:
plot_training(history)

### Evaluasi Model (train_df, valid_df, test_df)

In [ ]:
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

### Evaluasi lanjutan test_df

In [ ]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(y_pred)

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cmtrx = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cmtrx, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

## Menyimpan Model VGG19(fine tuned)

In [ ]:
model_name = model.input_names[0][:-6]
subject = 'crocidolomia-pavonana-vgg-19'
acc = test_score[1] * 100
save_path = ''

# Save model
save_id = str(f'{model_name}-{subject}-{"%.2f" %round(acc, 2)}.h5')
model_save_loc = os.path.join(save_path, save_id)
model.save(model_save_loc)
print(f'model was saved as {model_save_loc}')

# Save weights
weight_save_id = str(f'{model_name}-{subject}-weights.h5')
weights_save_loc = os.path.join(save_path, weight_save_id)
model.save_weights(weights_save_loc)
print(f'weights were saved as {weights_save_loc}')

## Import saved model

In [ ]:
model_path = '/content/i-crocidolomia-pavonana-resnet-v2-50-94.20.h5'

In [ ]:
model_load = tf.keras.models.load_model(model_path)

In [ ]:
model_layers = [layer.name for layer in model_load.layers]

In [ ]:
n = 4
img_path = test_df.iloc[n,0]
true_label = test_df.iloc[n,1]

img = keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
    # `array` is a float32 Numpy array of shape (299, 299, 3)
array = keras.preprocessing.image.img_to_array(img) / 255
    # We add a dimension to transform our array into a "batch"
array1 = np.expand_dims(array, axis=0)

In [ ]:
array1.shape

In [ ]:
array1[0][0][100]

In [ ]:
hasil = model_load.predict(array1)

In [ ]:
print(classes[np.argmax(hasil[0])])
print(f"True : {true_label}")

## Membuat GRAD-CAM
### Prediction and Bounding Box Function

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

### test heatmap

In [ ]:
model_layers[-10:]

In [ ]:
# VGG19s last convolution layer
last_conv_layer_name = model_layers[-8]
last_conv_layer_name

In [ ]:
# Print what the top predicted class is
preds = model_load.predict(array1)
# print("Predicted:", decode_predictions(preds, top=1)[0])
print(preds)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(array1, model_load, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
def image_to_heatmap_and_predict(img_path, model, ground_truth_label):
    # Prepare image
    img = keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    img_array = np.expand_dims(array, axis=0) / 255
        #img_array = preprocess_input(get_img_array(img_path, size=img_size)) # ISSUE
    # Print what the top predicted class is
    preds = model.predict(img_array)
    print(preds)
    predicted_label = classes[np.argmax(preds[0])]
    # print("Predicted:", decode_predictions(preds, top=1)[0])
    print(f"Prediksi           : {predicted_label}")
    print(f"Label Sesungguhnya : [{ground_truth_label}]")

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    return heatmap, predicted_label

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.5):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    (superimposed_img.save(cam_path))

    # Display Grad CAM
    display(Image(img_path))
    display(Image(cam_path))

    return '/content/'+ str(cam_path)

In [ ]:
def grad_cam_to_mask(img_path, cam_path):
    # Read the images
    img = cv2.imread(cam_path)
    img2 = cv2.imread(img_path)

    # Resizing the image
    image = cv2.resize(img, (700, 600))
    image2 = cv2.resize(img2, (700, 600))

    # Convert Image to Image HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Defining lower and upper bound HSV values
    lower = np.array([0, 100, 20])
    upper = np.array([35, 255, 255])

    # Defining mask for detecting color
    mask = cv2.inRange(hsv, lower, upper)

    # Bitwise-AND mask and original image
    result = cv2.bitwise_and(image2,image2, mask= mask)


    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.tight_layout()

    plt.axis('off')

    plt.savefig('heatmap_mask.jpg', dpi=300, bbox_inches='tight', pad_inches=0)
    heatmap_mask_path = '/content/heatmap_mask.jpg'

    return heatmap_mask_path

In [ ]:
def draw_bounding_area(img_path, heatmap_mask_path, ground_truth_label, label, output_filename):
    im = cv2.imread(heatmap_mask_path)
    im_xray = cv2.imread(img_path)

#     assert im is not None, "file could not be read, check with os.path.exists()"

    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    imgray = cv2.resize(imgray, (700, 600))
    im2 = cv2.resize(im_xray, (700, 600))
    # imgray2 = cv2.cvtColor(im_xray, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 20, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    result = cv2.drawContours(im2, contours, -1, (0,255,0), 3)

    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

    # save image
    plt.axis('off')

    prediction_text = f"Ground Truth: {label}, prediction: [{ground_truth_label}]"
    plt.title(prediction_text, color= 'blue', fontsize= 12)

    plt.savefig(output_filename, dpi=300, bbox_inches='tight', pad_inches=0)

    plt.show()

In [ ]:
def prediction(test_data_index, output_filename='bounding_crocidolomia_pavonana'):
    # image's path for prediction
    test_pict = test_df.iloc[test_data_index,:]
    img_path = test_pict.filepaths
    output_file = output_filename + '.jpg'

    # make prediction and heatmap
    heatmap, predict_label = image_to_heatmap_and_predict(img_path = img_path, model = model_load, ground_truth_label = test_pict.labels)

    # generate grad cam
    grad_cam_path = save_and_display_gradcam(img_path, heatmap)


    # make grad cam maks
    heatmap_mask_path = grad_cam_to_mask(img_path = img_path, cam_path = grad_cam_path)

    # draw bounding area
    draw_bounding_area(img_path = img_path, heatmap_mask_path = heatmap_mask_path, ground_truth_label = test_pict.labels, label = predict_label, output_filename=output_file)


In [ ]:
prediction(33)

In [ ]:
prediction(18)

In [ ]:
prediction(0)

In [ ]:
prediction(14)